# Representación semántica y búsqueda
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

Como anteriormente se presento, una de las representaciones semánticas más aceptadas son aquellas basadas en word embeddings. Recordando, estas son representaciones vectoriales de cada palabra, donde la semántica se asocia con la estructura en un espacio métrico, i.e., cercano en la métrica significa similar semánticamente.

# Representación semántica de documentos basada en _word embeddings_
Un documento puede verse como una bolsa de palabras, como en el modelado tradicional, pero en lugar de usar los términos como símbolos se pueden usar los vectores semánticos. El como usar esta nube de puntos multidimensional para obtener una representación computacionalmente manejable y a la vez eficaz, es un tema que ha llevado al desarrollo de modelos cada vez más complejos, como pueden ser los grandes modelos de lenguaje. 

La manera forma más directa de definir una representación semántica de documentos es el uso de centroides, esto es, un vector de la misma dimensión que sumariza a un conjunto de vectores usando su media geométrica. De manera más precisa, sea $E$ una matrix $m \times n$ de embeddings del vocabulario, i.e., considerando un vocabulario de tamaño $n$ y dimensión $m$.
Sea $D_{m,\ell}$ la submatriz de $E$ que contiene los $\ell$ vectores del documento $\textsf{doc}$. El prototipo $\vec{d}$ de $D$ esta definido de la siguiente forma:

\begin{equation}
\vec{d} = \frac{1}{\ell} \sum_{i=1}^\ell D_i
\end{equation}


Donde $D_i$ es el i-ésimo vector columna de $D$. Dado que se usa el coseno como similitud, también es factible el cálculo como la suma vectorial normalizada, esto es,

$$ \vec{d} = \frac{\sum_{i=1}^\ell D_i}{\lVert \sum_{i=1}^\ell D_i \rVert}$$

También es posible añadir información local basada en la representación de bolsa de plabras. Por ejemplo, realizar una suma pesada usando el peso de las palabras mediante la frecuencia normalizada de término (TF) o la probabilidad de término $pt$, ver unidad 3 para más información.

$$\vec{d} = \sum_{i=1}^{\ell} \textsf{TF}(t_i, \textsf{doc}) \cdot D_i$$

Donde $t_i$ es el $i$-ésimo término de $\textsf{doc}$. Note que el orden no se captura y solo es necesario para la notación.

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \max \left\{ \sum_{u \in U} \min_{x \in V} d(u, x), \sum_{v \in V} \min_{x \in U} d(v, x) \right\}$$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{TF}$

$$ H^\textsf{TF}_d(U, V) = \max \left\{ \sum_{u \in U} {\textsf{TF}(u, U) \min_{x \in V} d(u, x)}, \sum_{v \in V} {\textsf{TF}(v, V) \min_{x \in U} d(v, x)} \right\} $$

# Notas adicionales
La versión de $H_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [ ]:

using Pkg
Pkg.activate(".")
!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="MLDatasets", version="0.7"),
    PackageSpec(name="Plots"),
    PackageSpec(name="HypertextLiteral"),
    PackageSpec(name="Embeddings", version="0.4")
])

using SimilaritySearch, Plots, LinearAlgebra, Embeddings, HypertextLiteral


In [ ]:
function load_dataset()
    emb = load_embeddings(GloVe{:en}, 2)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end

    db = MatrixDatabase(emb.embeddings)
    db, emb.vocab
end

function create_index()
    db, vocab = load_dataset()
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index)
    optimize!(index, MinRecall(0.9))
    index, vocab
end


### El índice métrico se crea, además se toma el vocabulario

In [ ]:
@time index, vocab = create_index()
vocab2id = Dict(w => i for (i, w) in enumerate(vocab));

### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

In [ ]:
let k = 32
    @time I, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    @time gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    macrorecall(gI, I)
end

### Búsqueda y presentación de los resultados

In [ ]:
function search_and_display(index, vocab, q, res, k, qword)
    res = reuse!(res, k)
    @time search(index, q, res)

    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$(vocab[id])</td><td>$id</td><td>$(round(d, digits=3))</td></tr>")
    end

    display(@htl """<h2>resultados for "$qword"</h2>
    <table>
    <th>  <td>word</td> <td>id</td> <td>dist</td> </th>
        $L
    </table>
    """)
end

function search_and_display(index, vocab, qid::Integer, res, k=maxlength(res))
    search_and_display(index, vocab, index[qid], res, k, vocab[qid])
end

display(@htl "<h1>Ejemplos de búsqueda (aleatorios)</h1>")
res = KnnResult(7)
for i in 1:3
    for qid in rand(1:length(vocab))
        search_and_display(index, vocab, qid, res)
    end
end

In [ ]:
search_and_display(index, vocab, vocab2id["glove"], res, 15)

In [ ]:
# Analogias

In [ ]:
function analogy(a, b, c, k)
	v = index[vocab2id[a]] - index[vocab2id[b]] + index[vocab2id[c]]
	normalize!(v)
	search_and_display(index, vocab, v, res, k, "<$a> - <$b> + <$c>")
end

analogy("father", "man", "woman", 5)
analogy("fireman", "man", "woman", 5)
analogy("policeman", "man", "woman", 5)
analogy("mississippi", "usa", "france", 5)

# Actividades
- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implmentaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
